<a href="https://colab.research.google.com/github/MihneaGherghel/ColabCopy/blob/main/FakeNews3%2C0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import re
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
class OptainData:
  def __init__(self,file_path):
    self.data = pd.read_csv(file_path)
    self.data['label'] = pd.to_numeric(self.data['label'], errors='coerce')
    self.data = self.data.dropna()
    self.data['label'] = self.data['label'].astype(int)
    self.data.reset_index(inplace=True)

  def split_labels_samples(self):
    samples=self.data.drop('label',axis=1)
    labels=self.data['label']
    return samples,labels,self.data

In [4]:
data=OptainData('./gdrive/MyDrive/train.csv/FakeNews.csv')
samples,labels,samples_labels=data.split_labels_samples()

In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
import nltk
import numpy as np
import torch
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
messages=samples_labels.copy()
messages.reset_index(inplace=True)
corpus=[]
ps=PorterStemmer()
for i in range(0,len(messages)):
  review=re.sub('[^a-zA-Z]',' ', messages['title'][i])
  review=review.lower()
  review=review.split()
  review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
  review=' '.join(review)
  corpus.append(review)

In [7]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout

In [8]:
voc_size=5000
onehot_repr=[one_hot(words,voc_size) for words in corpus]

In [9]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

In [10]:
import numpy as np
from sklearn.model_selection import train_test_split

In [11]:
X_final=np.array(embedded_docs)
y_final=np.array(messages['label'])

In [12]:
X_train, X_test, y_train, y_test= train_test_split(X_final, y_final, test_size=0.33,random_state=42)

In [13]:
train_input_tensor = torch.from_numpy(X_train)
train_target_tensor = torch.from_numpy(y_train)
test_input_tensor = torch.from_numpy(X_test)
test_target_tensor = torch.from_numpy(y_test)

In [14]:
train_dataset = torch.utils.data.TensorDataset(train_input_tensor, train_target_tensor)
test_dataset = torch.utils.data.TensorDataset(test_input_tensor, test_target_tensor)

In [15]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=True)

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim

In [17]:
class LSTMFakeNewsClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(LSTMFakeNewsClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.linear = nn.Linear(hidden_dim, 1)

    def forward(self, text):
        embeds = self.embedding(text)
        lstm_out, _ = self.lstm(embeds)
        output = self.linear(lstm_out[:, -1])
        return output


In [18]:
model=LSTMFakeNewsClassifier(vocab_size=voc_size, embedding_dim=100, hidden_dim=258)

In [19]:
optimizer = optim.Adam(model.parameters(),lr=0.001)
criterion = nn.MSELoss()

In [ ]:
numarcazuricorecte=0
for epoch in range(5):
    for text,label in train_loader:
        # Zero the gradients
        label=label.float()
        optimizer.zero_grad()
        # Forward pass
        outputs = model(text).float()
        loss = criterion(outputs, label)
        # Backward pass
        loss.backward()
        optimizer.step()
        # Print progress
    correct=0
    for text,label in test_loader:
      outputs = model(text)
      predictions = outputs > 0.5
      labels = label > 0.5
      c = [labels[i]==predictions[i][0] for i in range(0,len(outputs),1)]
      for i in c:
        if i==True:
          correct+=1
    print( print('Epoch: [{}/{}], Correct: [{}/{}]'.format(
                epoch + 1, 5, len(X_test), correct)))
